## TFRecord 生成
### 一、为什么使用 TFRecord
正常情况下我们训练文件夹经常会生成 train, test 或者val文件夹，这些文件夹内部往往会存着成千上万的图片或文本等文件，这些文件被散列存着，这样不仅占用磁盘空间，并且再被一个个读取的时候会非常慢，繁琐。占用大量内存空间（有的大型数据不足以一次性加载）。此时我们TFRecord格式的文件存储形式会很合理的帮我们存储数据。TFRecord内部使用了“Protocol Buffer”二进制数据编码方案，它只占用一个内存块，只需要一次性加载一个二进制文件的方式即可，简单，快速，尤其对大型训练数据很友好。而且当我们的训练数据量比较大的时候，可以将数据分成多个TFRecord文件，来提高处理效率。
### 二、生成 TFRecord 简单实现方式
我们可以简单分成两个部分来介绍如何生成 TFRecord，分别是 TFRecord 生成器以及样本 Example 模块。
1. **TFRecord 生成器**


In [ ]:
writer = tf.python_io.TFRecordWriter(record_path)
writer.write(tf_example.SerializeToString())
writer.close()

这里面 *writer* 就是我们 *TFrecord* 生成器。接着我们就可以通过 *writer.write(tf_example.SerializeToString())* 来生成我们所要的 *tfrecord* 文件了。这里需要注意的是我们 *TFRecord* 生成器在写完文件后需要关闭 *writer.close()*。这里 *tf_example.SerializeToString()* 是将 *Example* 中的 *map* 压缩为二进制文件，更好的节省空间。那么 *tf_example* 是如何生成的呢？那就是下面所要介绍的样本 *Example* 模块了。


2. **Example 模块**  
首先来看一下Example 协议是什么样子的

In [ ]:
message Example {
  Features features = 1;
};

message Features {
  map<string, Feature> feature = 1;
};

message Feature {
  oneof kind {
    BytesList bytes_list = 1;
    FloatList float_list = 2;
    Int64List int64_list = 3;
  }
};

我们可以看出上面的tf_example可以写入的数据形式有三种，分别是BytesList, FloatList以及Int64List的类型。那我们如何写一个tf_example呢？下面有一个简单的例子。

In [ ]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

tf_example = tf.train.Example(
        features=tf.train.Features(feature={
            'image/encoded': bytes_feature(encoded_jpg),
            'image/format': bytes_feature('jpg'.encode()),
            'image/class/label': int64_feature(label),
            'image/height': int64_feature(height),
            'image/width': int64_feature(width)}))

下面我们来好好从外部往内部分解来解释一下上面的内容。  
（1）tf.train.Example(features = None) 这里的 features 是 tf.train.Features 类型的特征实例。  
（2）tf.train.Features(feature = None) 这里的 feature 是以字典的形式存在，*key：要保存数据的名字     value：要保存的数据，但是格式必须符合tf.train.Feature实例要求。
